In [1]:
from const import CONSTS

import util
import tiger_xml_to_const
import converter

from tiger_xml_to_const import ConstituentTree, DependencyTree
import xml.etree.ElementTree as ET
import pickle

import importlib
importlib.reload(util)
importlib.reload(tiger_xml_to_const)
importlib.reload(converter)


<module 'converter' from '/home/james/programming/ml/german/german_parser/util/converter.py'>

In [2]:
with open(f"{CONSTS['data_dir']}/tiger/tiger_2.2_utf8.xml", "rb") as f:
    document = ET.parse(f)

In [190]:
all_s = document.findall(".//s")

In [191]:
# errors = 0
# has_unary = 0
# has_empty_verb_constituents = 0

# cannot_use = 0

# error_messages = []

# sents = []
# for idx, s in enumerate(all_s):
#     print(f"Parsing sentence {idx + 1}; errors {errors}")
#     s_cannot_use = False

#     try:
#         sent = ConstituentTree(s)
#         has_unary += int(sent.has_unary)
#         has_empty_verb_constituents += int(sent.has_empty_verb_constituents)

#         s_cannot_use = sent.has_unary or sent.has_empty_verb_constituents

#         sent.check_constituent_rules()
#     except Exception as e:
#         errors += 1
#         error_messages.append(f"sentence id {idx + 1} has error {e}")

#         s_cannot_use = True
#         # break

#     if not s_cannot_use:
#         sents.append(sent)
#     cannot_use += int(s_cannot_use)

In [192]:
s_test = ConstituentTree.from_tiger_xml(all_s[26])

In [193]:
d_tree = DependencyTree.from_c_tree(s_test)

In [194]:
c_tree = ConstituentTree.from_d_tree(d_tree)

In [195]:
s_test.terminals

{1: Terminal(word='Ihm', lemma='ihm', pos='PPER', morph='3.Dat.Sg.Masc', case='Dat', number='Sg', gender='Masc', person='3', degree='--', tense='--', mood='--', idx=1),
 2: Terminal(word='gelingt', lemma='gelingen', pos='VVFIN', morph='3.Sg.Pres.Ind', case='--', number='Sg', gender='--', person='3', degree='--', tense='Pres', mood='Ind', idx=2),
 3: Terminal(word='es', lemma='es', pos='PPER', morph='3.Nom.Sg.Neut', case='Nom', number='Sg', gender='Neut', person='3', degree='--', tense='--', mood='--', idx=3),
 4: Terminal(word='aber', lemma='aber', pos='ADV', morph='--', case='--', number='--', gender='--', person='--', degree='--', tense='--', mood='--', idx=4),
 5: Terminal(word='nicht', lemma='nicht', pos='PTKNEG', morph='--', case='--', number='--', gender='--', person='--', degree='--', tense='--', mood='--', idx=5),
 6: Terminal(word=',', lemma='--', pos='$,', morph='--', case='--', number='--', gender='--', person='--', degree='--', tense='--', mood='--', idx=6),
 7: Terminal(wo

In [199]:
c_tree.draw()

(<toyplot.canvas.Canvas at 0x7f10519905d0>,
 <toytree.Render.ToytreeMark at 0x7f11c48a3c10>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="600.0px" viewBox="0 0 600.0 600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t95766191b90840fc971c97c6c4b284c8"> 19.ADJA 18.PIAT 17.KOKOM 20.NN 15.ADV 14.NE 13.APPR 11.ART 12.NN 10.PRELS 16.VVFIN 7.ART 8.NN 23.NN 22.APPR 24.PTKZU 25.VVINF 3.PPER 5.PTKNEG 4.ADV 1.PPER 2.VVFIN 26.$. 21.$COMMA 9.$COMMA 6.$COMMA NP PP AVP NP S NP PP VZ VP NP S VROOT andere jeder als Konzern mehr Washington in der Regierung der ähnelt den Koloß Schwung in zu bringen es nicht aber Ihm gelingt . , , ,

In [204]:
[t.attrib["word"] for t in document.findall(".//t")]

['``',
 'Ross',
 'Perot',
 'wäre',
 'vielleicht',
 'ein',
 'prächtiger',
 'Diktator',
 "''",
 'Konzernchefs',
 'lehnen',
 'den',
 'Milliardär',
 'als',
 'US-Präsidenten',
 'ab',
 '/',
 'Texaner',
 'gibt',
 'nur',
 'vage',
 'Auskunft',
 'über',
 'seine',
 'Wirtschaftspolitik',
 'Der',
 'texanische',
 'Milliardär',
 'Ross',
 'Perot',
 'hat',
 'das',
 'politische',
 'Establishment',
 'in',
 'Washington',
 'aufgeschreckt',
 '.',
 'Nach',
 'Meinungsumfragen',
 'liegt',
 'der',
 'parteilose',
 'Self-mademan',
 'gut',
 'im',
 'Rennen',
 'um',
 'den',
 'Chefsessel',
 'im',
 'Weißen',
 'Haus',
 'mit',
 'dem',
 'amtierenden',
 'Präsidenten',
 'George',
 'Bush',
 'und',
 'dem',
 'Demokraten',
 'Bill',
 'Clinton',
 '.',
 'Daß',
 'Perot',
 'ein',
 'Unternehmen',
 'erfolgreich',
 'leiten',
 'kann',
 ',',
 'davon',
 'sind',
 'selbst',
 'seine',
 'Kritiker',
 'überzeugt',
 '.',
 'Ob',
 'diese',
 'Fähigkeiten',
 'aber',
 'ausreichen',
 ',',
 'um',
 'die',
 'größte',
 'Volkswirtschaft',
 'der',
 'Welt',

In [209]:
! explorer.exe .